In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
!pip install transformers==4.33.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.33.1 which is incompatible.


In [6]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=179b468d021eb9e36c299ecf9b7fb282cc46db5d2831936ec87b702e598b89c6
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [7]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [8]:
!pip install huggingface_hub

In [9]:
!pip show transformers

Name: transformers
Version: 4.33.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [10]:
# !GIT_SSH_COMMAND="ssh -i /path/to/your/ssh_key" git clone git@hf.co:username/model-repo.git && cd model-repo && git lfs pull

In [11]:
!git lfs install

Git LFS initialized.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [12]:
!git clone https://huggingface.co/facebook/layerskip-llama2-7B

fatal: destination path 'layerskip-llama2-7B' already exists and is not an empty directory.


In [13]:
import torch
from draftandverify.modeling_llama import LlamaForCausalLM
from transformers import AutoTokenizer
from draftandverify.decoding import clip_input, infer_input_ids
from rouge_score import rouge_scorer
import numpy as np
from datasets import load_dataset

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


(Re-)Loading modeling...


In [ ]:
torch.nn.Linear.reset_parameters = lambda x: None
model = LlamaForCausalLM.from_pretrained('layerskip-llama2-7B', torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained('layerskip-llama2-7B')

# model = LlamaForCausalLM.from_pretrained('meta-llama/Llama-2-7b-hf', torch_dtype=torch.bfloat16, token="")
# tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf', token="")

device='cuda:0'

model = model.to(device).eval()

SEED = 42

In [16]:
_attn_skip_layer_id_set, _mlp_skip_layer_id_set = model.get_skip_layers()
print(_attn_skip_layer_id_set, _mlp_skip_layer_id_set)

[2, 4, 9, 10, 12, 15, 18, 19, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31] [2, 8, 9, 13, 15, 16, 22, 23, 27, 29, 30, 31]


In [17]:
np.random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
n_shot = 1
task_name = 'cnndm'
prompt_shots = ''

data = load_dataset('cnn_dailymail', name='3.0.0', split='test') .shuffle(seed=SEED).select(range(50))
shots = load_dataset('cnn_dailymail', name='3.0.0', split='train').shuffle(seed=SEED).select(range(n_shot))
prompt_keys=['article','highlights']

for i in range(n_shot):
    prompt = 'Article: ' + shots[i][prompt_keys[0]] + '\nSummary: ' + shots[i][prompt_keys[1]].replace('\n', '') + '\n'
    prompt_shots += prompt

In [19]:
print(len(data))

50


In [ ]:
metrics = []
rouge = rouge_scorer.RougeScorer(['rouge2'], use_stemmer=True)

main_metrics = {
    'rouge2_base':[],
    'rouge2_essg2':[], 
    'rouge2_essg_autoth':[],
    'time_base':[],
    'time_essg2':[], 
    'time_essg_autoth':[],
    'token_time_base':[],
    'token_time_essg2':[], 
    'token_time_essg_autoth':[],
    'matchness_essg2':[],
    'num_drafted_tokens_essg2':[],
    'matchness_essg_autoth':[],
    'num_drafted_tokens_essg_autoth':[]
}

In [ ]:
# benchmarking implementation adapted from:
# https://github.com/dilab-zju/self-speculative-decoding/tree/main

for i, x in enumerate(data):
    input_ids = clip_input(tokenizer, x, task_name, max_new_tokens=512,prompt_shots=prompt_shots)

    if i == 0:
        th_stop_draft_essg2 = 0.40
        th_stop_draft_essg_autoth = 0.60
    else:
        th_stop_draft_essg2 = result_essg2['th_stop_draft']
        th_stop_draft_essg_autoth = result_essg_autoth['th_stop_draft']
    result_base = infer_input_ids(model, tokenizer, input_ids, generate_fn='base',
                max_new_tokens=512, do_sample=False, early_stop=True)
    result_essg2 = infer_input_ids(model, tokenizer, input_ids, generate_fn='essg',
                max_new_tokens=512, early_stop=True, max_step_draft=12,
                th_stop_draft=th_stop_draft_essg2, auto_th_stop_draft=False,
                do_sample=False, do_sample_draft=False)
    result_essg_autoth = infer_input_ids(model, tokenizer, input_ids, generate_fn='essg',
                max_new_tokens=512, early_stop=True, max_step_draft=12,
                th_stop_draft=th_stop_draft_essg_autoth, auto_th_stop_draft=True, auto_parameters=[1,0.50,0.90,1e-2,0.90],
                do_sample=False, do_sample_draft=False)

    if len(result_base['completion']) < 5 or ('.....' in result_base['completion'][:5]):
        print("too short, skip")
        continue

    references = x['highlights']

    results = [
        ('base', result_base),
        ('essg2', result_essg2),
        ('essg_autoth', result_essg_autoth)
    ]

    for key, result in results:
        main_metrics['time_' + key].append(result['time'])
        main_metrics['token_time_' + key].append(result['time'] / result['generate_ids'].shape[1])
        if key != 'base':
            main_metrics['matchness_' + key].append(result['matchness'])
            main_metrics['num_drafted_tokens_' + key].append(result['num_drafted_tokens'])
        clip_pred = result['completion'].find("\nArticle:")
        if clip_pred > 0:
            prediction = result['completion'][:clip_pred]
        else:
            prediction = result['completion']
        rouge_score = rouge.score(prediction, references)
        main_metrics['rouge2_' + key].append(rouge_score['rouge2'].fmeasure)

    metric = {
        'mean rouge-2 base':np.mean(main_metrics['rouge2_base']),
        f'mean rouge-2 essg th {th_stop_draft_essg2}':np.mean(main_metrics['rouge2_essg2']),
        'mean rouge-2 essg autoth':np.mean(main_metrics['rouge2_essg_autoth']),
        'mean time base':np.mean(main_metrics['time_base']),
        f'mean time essg th {th_stop_draft_essg2}':np.mean(main_metrics['time_essg2']),
        'mean time essg autoth':np.mean(main_metrics['time_essg_autoth']),
        f'E2E mean speed up essg th {th_stop_draft_essg2}':np.mean(main_metrics['time_base'])/np.mean(main_metrics['time_essg2']),
        'E2E mean speed up essg autoth':np.mean(main_metrics['time_base'])/np.mean(main_metrics['time_essg_autoth']),
        'mean token time base':np.mean(main_metrics['token_time_base']),
        f'mean token time essg th {th_stop_draft_essg2}':np.mean(main_metrics['token_time_essg2']),
        'mean token time essg autoth':np.mean(main_metrics['token_time_essg_autoth']),
        f'E2E mean token speed up essg th {th_stop_draft_essg2}':np.mean(main_metrics['token_time_base'])/np.mean(main_metrics['token_time_essg2']),
        'E2E mean token speed up essg autoth':np.mean(main_metrics['token_time_base'])/np.mean(main_metrics['token_time_essg_autoth']),
        f'mean matchness essg th {th_stop_draft_essg2}':np.mean(main_metrics['matchness_essg2']),
        'mean matchness essg autoth':np.mean(main_metrics['matchness_essg_autoth']),
        f'mean num_drafted_tokens essg th {th_stop_draft_essg2}':np.mean(main_metrics['num_drafted_tokens_essg2']),
        'mean num_drafted_tokens essg autoth':np.mean(main_metrics['num_drafted_tokens_essg_autoth']),
    }
    for key, value in metric.items():
        if isinstance(value, float):
            metric[key] = f"{value:.4f}"

    metrics.append(metric)

In [22]:
len(metrics)

50

In [41]:
# regular llama

for key in metrics[0].keys():
    v = 0
    for m in metrics:
        v += float(m[key])
    v /= 50
    print(key, v)

mean rouge-2 base 0.037966
mean rouge-2 essg th 0.4 0.037871999999999996
mean rouge-2 essg autoth 0.039973999999999996
mean time base 20.665521999999996
mean time essg th 0.4 14.950458000000003
mean time essg autoth 15.224526000000001
E2E mean speed up essg th 0.4 1.3823219999999994
E2E mean speed up essg autoth 1.3577860000000002
mean token time base 0.040369999999999996
mean token time essg th 0.4 0.029204000000000035
mean token time essg autoth 0.029744000000000013
E2E mean token speed up essg th 0.4 1.3823219999999994
E2E mean token speed up essg autoth 1.3577860000000002
mean matchness essg th 0.4 0.9019200000000002
mean matchness essg autoth 0.9023260000000001
mean num_drafted_tokens essg th 0.4 466.423368
mean num_drafted_tokens essg autoth 447.99715599999996


In [23]:
# layerskip llama

for key in metrics[0].keys():
    v = 0
    for m in metrics:
        v += float(m[key])
    v /= 50
    print(key, v)

mean rouge-2 base 0.053758000000000014
mean rouge-2 essg th 0.4 0.05575399999999999
mean rouge-2 essg autoth 0.055504000000000005
mean time base 21.569596000000004
mean time essg th 0.4 13.542060000000003
mean time essg autoth 13.635637999999998
E2E mean speed up essg th 0.4 1.592806
E2E mean speed up essg autoth 1.5818020000000006
mean token time base 0.04212600000000004
mean token time essg th 0.4 0.026447999999999996
mean token time essg autoth 0.02663599999999999
E2E mean token speed up essg th 0.4 1.592806
E2E mean token speed up essg autoth 1.5818020000000006
mean matchness essg th 0.4 0.8867919999999998
mean matchness essg autoth 0.9152680000000001
mean num_drafted_tokens essg th 0.4 492.66797599999995
mean num_drafted_tokens essg autoth 453.7901599999999
